In [2]:
import pandas as pd
import numpy as np
import heapq
import datetime

In [3]:
puzzle_info = pd.read_csv('./input_data/puzzle_info.csv')
puzzles = pd.read_csv('./input_data/puzzles.csv')

In [4]:
def to_array(state):
    return np.array(state.split(';'))

def to_string(state):
    return ";".join(state)

def heuristic(state, goal_state):
    return np.sum(np.where(state != goal_state, 1, 0))

def gen_neighbors(state, moves):
    neighbors = []
    for move_id, move in moves.items():
        inv = np.arange(len(move))
        for i, v in enumerate(move):
            inv[v] = i
        neighbors.append([f'-{move_id}', state[inv]])
        neighbors.append([move_id, state[move]])
    return neighbors

def a_star_solve(goal_state, init_state, moves, max_time=2):
    priority_queue = []
    track = []
    state_cost = {init_state: 0}

    goal_state_array = to_array(goal_state)
    init_state_array = to_array(init_state)

    heapq.heappush(priority_queue, (0 + heuristic(init_state_array, goal_state_array), init_state, track))

    t = datetime.datetime.now()
    time_passed = (datetime.datetime.now() - t).seconds / 60
    while priority_queue or time_passed < max_time:
        current_f_cost, current_state, track = heapq.heappop(priority_queue)
        current_state_array = to_array(current_state)

        if current_state == goal_state:
            return track
        
        for neighbor in gen_neighbors(current_state_array, moves):
            new_g_cost = len(track) + 1
            neighbor_string = to_string(neighbor[1])
            if neighbor_string not in state_cost or new_g_cost < state_cost[neighbor_string]:
                state_cost[neighbor_string] = new_g_cost
                f_cost = new_g_cost + heuristic(neighbor[1], goal_state_array)
                node_track = track + [neighbor[0]]
                heapq.heappush(priority_queue, (f_cost, neighbor_string, node_track))

        time_passed = (datetime.datetime.now() - t).seconds / 60
        
    return None


class IDA:
    def __init__(self):
        self.seen_states = {}

    def solve(self, init_state, goal_state, moves, max_time=2):
        threshold = heuristic(init_state, goal_state)
        track = []
        self.seen_states[to_string(init_state)] = 0

        t = datetime.datetime.now()
        time_passed = (datetime.datetime.now() - t).seconds / 60
        while time_passed < max_time:
            temp, track = self.search(init_state, 0, threshold, moves, goal_state, track)
            if temp == "FOUND":
                return track
            #if temp == np.inf:
            #    return "NOT_FOUND"
            threshold = temp
            time_passed = (datetime.datetime.now() - t).seconds / 60

    def search(self, node, g, threshold, moves, goal_state, track):
        f = g + heuristic(node, goal_state)
        if f > threshold:
            return f, track
        if to_string(node) == to_string(goal_state):
            return "FOUND", track
        min_threshold = np.inf
        for child in gen_neighbors(node, moves):
            child_string = to_string(child[1])
            if not child_string in self.seen_states or len(track) < self.seen_states[child_string]:
                self.seen_states[child_string] = len(track)
                new_track = track + [child[0]]
                temp, new_track = self.search(child[1], g + 1, threshold, moves, goal_state, new_track)
                if temp == "FOUND":
                    return "FOUND", new_track
                if temp < min_threshold:
                    min_threshold = temp
        return min_threshold, track


In [10]:
def get_problem(puzzle_id):
    puzzle = puzzles.loc[puzzle_id]
    
    # dtype string
    goal_state = puzzle['solution_state']
    init_state = puzzle['initial_state']

    # dtype string
    puzzle_type = puzzle['puzzle_type']

    # eval to convert string to dict
    moves = eval(puzzle_info[puzzle_info['puzzle_type'] == puzzle_type]['allowed_moves'].item())
    moves = {move_id: np.array(move) for move_id, move in moves.items()}

    return [goal_state, init_state, moves]

t_all = datetime.datetime.now()
for i in range(329, 330):
    print(f'\nProblem: {i}')
    vars = get_problem(i)
    goal_state, init_state, moves = vars[0], vars[1], vars[2]
    t = datetime.datetime.now()
    solution = a_star_solve(goal_state, init_state, moves, max_time=60)
    
    #solver = IDA()
    #solution = solver.solve(to_array(init_state), to_array(goal_state), moves)
    
    print(solution)
    print((datetime.datetime.now() - t))
print(f'\nOverall time: {(datetime.datetime.now() - t_all)}')


Problem: 329
['-r', 'l', '-r', '-l', '-l', '-r', '-r', 'l', 'l', 'l', 'l', '-r', '-r', '-r', 'l', 'l', 'l', '-r', '-r', 'l', 'l', 'l', '-r', '-r', '-r', '-r', 'l', 'l', 'r']
0:05:07.938953

Overall time: 0:05:07.941873
